In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font family: Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family: Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding top:120px;}
div.text_cell_render ul li {font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font color="red" size="6"> 11장. 데이터프레임과 시리즈(Pandas) </font></b>
# 9절. 데이터 그룹화 및 집계
- std 종별 표준편차값

In [4]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Groupby
cf. pivot_table

#### 1) 그룹화 방법1

In [16]:
# 1) 그룹화 방법1
#      df.groupby(by=[그룹핑할열(여러개가능)]).기초통계함수(min,max,mean이런거)[함수적용해서 출력할 열들]
#      df.groupby(by=[그룹핑할열(여러개가능)])[함수적용해서 출력할 열들].기초통계함수(min,max이런거)
#                                             [함수적용해서 출력할 열들] 생략시 그룹핑할 열을 제외한 모든 열 적용
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index()) #.head() 처음 5개만 보여줘 .sampe(숫자) 숫자 가져와서 랜덤하게 섞어 / reset_index()

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
10,5.4,3.7,1.5,0.2,setosa
36,5.5,3.5,1.3,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
60,5.0,2.0,3.5,1.0,versicolor
83,6.0,2.7,5.1,1.6,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
127,6.1,3.0,4.9,1.8,virginica
148,6.2,3.4,5.4,2.3,virginica


In [10]:
iris.groupby(by='Species').mean()[['Petal.Length','Petal.Width']]
iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean()

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [24]:
iris_grouped_sum =iris.groupby('Species').median()
#iris_grouped_sum.reset_index()             #inplace=True
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [26]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [22]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [28]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') |
                 (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


#### 2) 그룹화 방법2

In [29]:
#      df.groupby(by=[그룹핑할열(여러개가능)])[함수적용해서 출력할 열들].기초통계함수(min,max이런거)
#                                             [함수적용해서 출력할 열들] 생략시 그룹핑할 열을 제외한 모든 열 적용
# 2) 그룹화 방법2
#      df.pivot_table(index=[그룹핑할열(여러개가능)],
#                     values=[함수적용해서 출력할 열들],
#                     aggfunc=함수(기본값:mean))

In [32]:
iris.groupby('Species')['Petal.Length','Petal.Width'].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length','Petal.Width'],
                aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [33]:
iris.groupby('Species').count() # 결측치를 제외한 값의 갯수

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [38]:
iris.pivot_table(index='Species', aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [39]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### 다중열로 그룹화

In [40]:
# iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num 필드)
iris.shape # 데이터가 몇개 들었지?

(150, 5)

In [42]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [45]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.head()
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


#### 각열의 임의의 데이터에 결측치를 할당

In [ ]:
# 각열의 임의의 데이터에 결측치를 할당
#iris.iloc[0, 0] = np.nan  # iris에 0행 0열에 결측치 넣기

In [53]:
import random
print(random.sample(range(149), 3))

#iris.iloc[random.sample(range(149), 2), 1] = np.nan
#iris.iloc[random.sample(range(149), 3), 2] = np.nan
#iris.iloc[random.sample(range(149), 4), 3] = np.nan# 0열도 결측치 넣어

for col in range(iris.shape[1]-2): # iris.shape[1]-2 : 4
    iris.iloc[random.sample(range(150), col+1), col] = np.nan

[105, 8, 17]


In [54]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  149 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [59]:
iris.groupby(by=['Species','num'], as_index=False)[['Petal.Length', 'Petal.Width']].count()

,Species,num,Petal.Length,Petal.Width
0,setosa,0,24,25
1,setosa,1,25,25
2,versicolor,2,24,25
3,versicolor,3,25,23
4,virginica,4,24,24
5,virginica,5,25,24


In [60]:
iris.pivot_table(index=['Species','num'],
                values=['Petal.Length','Petal.Width'],
                aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              24           25
           1              25           25
versicolor 2              24           25
           3              25           23
virginica  4              24           24
           5              25           24

In [62]:
iris.groupby('Species').describe().T

Species                setosa  ...  virginica
Sepal.Length count  49.000000  ...  50.000000
             mean    5.006122  ...   6.588000
             std     0.356141  ...   0.635880
             min     4.300000  ...   4.900000
             25%     4.800000  ...   6.225000
             50%     5.000000  ...   6.500000
             75%     5.200000  ...   6.900000
             max     5.800000  ...   7.900000
Sepal.Width  count  49.000000  ...  49.000000
             mean    3.434694  ...   2.957143
             std     0.379995  ...   0.302765
             min     2.300000  ...   2.200000
             25%     3.200000  ...   2.800000
             50%     3.400000  ...   3.000000
             75%     3.700000  ...   3.100000
             max     4.400000  ...   3.800000
Petal.Length count  49.000000  ...  49.000000
             mean    1.463265  ...   5.546939
             std     0.175231  ...   0.556440
             min     1.000000  ...   4.500000
             25%     1.400000  ...   5.100000
             50%     1.500000  ...   5.500000
             75%     1.600000  ...   5.900000
             max     1.900000  ...   6.900000
Petal.Width  count  50.000000  ...  48.000000
             mean    0.246000  ...   2.022917
             std     0.105386  ...   0.276943
             min     0.100000  ...   1.400000
             25%     0.200000  ...   1.800000
             50%     0.200000  ...   2.000000
             75%     0.300000  ...   2.300000
             max     0.600000  ...   2.500000
num          count  50.000000  ...  50.000000
             mean    0.500000  ...   4.500000
             std     0.505076  ...   0.505076
             min     0.000000  ...   4.000000
             25%     0.000000  ...   4.000000
             50%     0.500000  ...   4.500000
             75%     1.000000  ...   5.000000
             max     1.000000  ...   5.000000

[40 rows x 3 columns]

## 9.2 레이블 인코딩

In [65]:
iris.drop('num', axis=1, inplace=True)
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,NaN,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [66]:
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,NaN,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


In [69]:
# 레이블 인코딩을 지원하는 클래스 import(LabelEncoder sklearn.preprocenssing)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.Species) # 지정한 열을 숫자로 바꿔줌
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,NaN,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포멧 <-> 롱포멧
- melt()를 이용한 언피벗팅
- 판다스 공식 문서의 melt : [pandas/pydata/reshaping](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt)
- Tidy Data(7page) : [Tidy Data 란](https://vita.had.co.nz/papers/tidy-data.pdf )

In [73]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality', 
                                   package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포멧 -> 롱포멧)

In [83]:
air_melted = airquality.melt(id_vars=['Month', 'Day'], 
                                        var_name='변수명', # 기본값 : variable
                                        value_name='값') # 기본값 : value

air_melted.sort_values(by=['Month','Day'], inplace=True)
air_melted.iloc[::20]

,Month,Day,변수명,값
0,5,1,Ozone,41.0
5,5,6,Ozone,28.0
10,5,11,Ozone,7.0
15,5,16,Ozone,14.0
20,5,21,Ozone,1.0
25,5,26,Ozone,NaN
30,5,31,Ozone,37.0
35,6,5,Ozone,NaN
40,6,10,Ozone,39.0
45,6,15,Ozone,NaN


#### melt 연습하기

In [86]:
# melt 연습하기 (부동산 와이드포멧 데이터를 롱포멧으로 변환)
%ls r'C:\ai_x\download\shareData\부동산_250213\'

매개 변수 포맷이 틀립니다 - "_x".


#### 11절에서 new_df

In [89]:
import pandas as pd
df = pd.read_csv('C:/ai_x/download/shareData/부동산_250213/전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv', encoding='cp949')
df.iloc[::10]

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,2014년9월,2014년10월,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,19404,19759,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443
10,충북,6589,6589,6611,6625,6678,6598,6587,6586,6586,6584,6529,6724,6743,6749,6747,6783,6790,6805,6682,6601,6603


In [98]:
new_df = df.melt(id_vars=['지역'], var_name='연도월', value_name='평당분양가') # 기본값 : value
#new_df.sort_values(by=['연도월','평당분양가'], inplace=True)
new_df.iloc[::20]

,지역,연도월,평당분양가
0,서울,2013년12월,18189
20,인천,2014년1월,10204
40,울산,2014년2월,8090
60,강원,2014년3월,6141
80,전북,2014년4월,6277
100,경남,2014년5월,6610
120,부산,2014년7월,9457
140,광주,2014년8월,7622
160,경기,2014년9월,10501
180,충북,2014년10월,6529


## 10.2 pivot_table을 이용한 피벗팅(롱포멧 -> 와이드포멧)

#### air_melted를 와이드 포멧으로 변환

In [101]:
# air_melted를 와이드 포멧으로 변환
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0


In [104]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                                        columns=['변수명'],
                                        values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

In [106]:
airquality2.head()

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3


# 11절. 데이터프레임(시리즈)에 함수적용시키기
- apply(func, axis=) : 축요소별(요소별) 함수 적용(데이터프레임이나 시리즈에서 가능) axis=0 0행부터 끝행까지 수행
- applymap(func) : 각 요소별 함수 적용(데이터 프레임에서만 가능, 2차원만가능)
- map(func, 리스트(시리즈)) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)
## 11.1 apply(func, axis=)

In [112]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [117]:
def get_year(datestr):
    return int(datestr.split('-')[0])

In [120]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [121]:
get_year('1999-05-01')

1999

In [122]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [124]:
get_month('1999-05-01')

5

In [128]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [131]:
X = iris.iloc[:, :-2]
# X의 열별 평균
avg = X.mean(axis=0) # 시리즈(1차원)
avg

Sepal.Length    5.848993
Sepal.Width     3.052027
Petal.Length    3.758503
Petal.Width     1.187671
dtype: float64

In [132]:
X.iloc[0] - avg

Sepal.Length   -0.748993
Sepal.Width     0.447973
Petal.Length         NaN
Petal.Width    -0.987671
dtype: float64

In [133]:
# X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.75,0.45,NaN,-0.99
1,-0.95,-0.05,-2.36,-0.99
2,-1.15,0.15,-2.46,-0.99
3,-1.25,0.05,-2.26,-0.99
4,-0.85,0.55,-2.36,-0.99
...,...,...,...,...
145,0.85,-0.05,1.44,NaN
146,0.45,-0.55,1.24,0.71
147,0.65,-0.05,1.44,0.81
148,0.35,0.35,1.64,1.11


## 11.2 applymap(func)

In [135]:
import numpy as np
X.applymap(np.round)

C:\Users\Admin\AppData\Local\Temp\ipykernel_15572\789488462.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X.applymap(np.round)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,NaN,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,NaN
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map(func, 리스트(시리즈))
- 1차원

In [137]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1])) # 0번째부터 끝번째까지 다해
member

,Name,Age,Email,Address,Birth,year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [ ]:
member.drop('Birth')

#### 연습하기(apply, map)

In [145]:
#new_df['연도월'].str.split('년')
# new_df의 연도월 데이터를 연도와 월을 int로 분리하여 컬럼을 추가하세요
new_df.head()
new_df['연도'] = new_df['연도월'].map(lambda x: int(x.split('년')[0]))
new_df['월'] = new_df['연도월'].map(lambda x: int(x.split('년')[1].replace('월','')))
new_df.head()

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


In [141]:
new_df.head()

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [150]:
new_df.drop(['연도','월'], axis=1, inplace=True)
new_df.head()

KeyError: "['연도', '월'] not found in axis"

In [148]:
def split_date(df):
    df = df.copy()
    df['연도']= int(df['연도월'].split('년')[0])
    df['월']= int(df['연도월'].split('년')[1][:-1])
    return df

In [149]:
split_date(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [147]:
# ★ 2차원 데이터 프레임의 apply 이용하는 방법

In [162]:
def year_month_append(df):
    'df를 받아 연도와 월 추가후 return'
    df = df.copy() # 깊은 복사
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [163]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [165]:
new_df = new_df.apply(year_month_append, axis=1)
new_df.head()

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


# 12절. 일괄변경하기(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [167]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                   columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


#### 모든 결측치를 0으로 대체

In [169]:
# 모든 결측치를 0으로 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [170]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

C:\Users\Admin\AppData\Local\Temp\ipykernel_15572\1193302488.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill')


,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [171]:
# 결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

C:\Users\Admin\AppData\Local\Temp\ipykernel_15572\4039330315.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill')


,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [173]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [174]:
# 결측치를 열별 평균값으로 대체
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [176]:
# 결측치 대체를 2번째 결측치까지만 대체
print(values)
df.fillna(value=values, limit=2)

{'A': 85, 'B': 80, 'C': 75, 'D': 90}


,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


## 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용